In [8]:
from docxtpl import DocxTemplate

In [10]:
import pandas as pd

In [12]:
import os

In [14]:
from docx2pdf import convert

In [16]:
from pathlib import Path

In [65]:
# 先单独测试文件是否存在
import os
file_path = 'D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/university.xlsx'
print("文件是否存在:", os.path.exists(file_path))

文件是否存在: True


In [67]:
# 先单独测试文件是否存在
import os
file_path = 'D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/research_areas.xlsx'
print("文件是否存在:", os.path.exists(file_path))

文件是否存在: True


In [127]:
def load_universities_data(file_path='D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/university.xlsx'):
    """
    加载大学列表数据
    """
    try:
        df = pd.read_excel(file_path)
        universities = df['university'].tolist()
        print(f"✅ 成功加载 {len(universities)} 所大学")
        print("📋 大学列表预览:")
        for i, uni in enumerate(universities[:5], 1):
            print(f"  {i}. {uni}")
        if len(universities) > 5:
            print(f"  ... 和 {len(universities) - 5} 所更多大学")
        return universities
    except FileNotFoundError:
        print(f"文件未找到：{file_path}")
        return []
    except Exception as e:
        print(f"加载大学数据失败：{e}")
        return []

In [57]:
print("开始调用函数...")
result = load_universities_data()  # 调用函数
print("函数执行完成，返回结果:", result)

开始调用函数...
Excel文件中的实际列名: ['university']
数据预览:
                                    university
0                           Harvard University
1  Massachusetts Institute of Technology (MIT)
2                        University of Chicago
3                         Princeton University
4                          Stanford University
√ 使用列名 'university' 成功加载30所大学
√ 大学列表预览:
1. Harvard University
2. Massachusetts Institute of Technology (MIT)
3. University of Chicago
4. Princeton University
5. Stanford University
...和25所更多大学
函数执行完成，返回结果: ['Harvard University', 'Massachusetts Institute of Technology (MIT)', 'University of Chicago', 'Princeton University', 'Stanford University', 'Brown University', 'Boston University', 'University of Michigan', 'University College London (UCL)', 'Duke University', 'University of Toronto', 'Cornell University', 'Johns Hopkins University', 'University of Nottingham', 'University of California-Irvine', 'Aarhus Universitet', 'Vrije Universiteit Amsterdam', 'University

In [71]:
import os
file_path = 'D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/research_areas.xlsx'
print("文件是否存在:", os.path.exists(file_path))

文件是否存在: True


In [195]:
try:
    df = pd.read_excel(file_path)
    print("文件读取成功")
    print("列名:", df.columns.tolist())
    print("前5行数据:")
    print(df.head())
except Exception as e:
    print(f"读取失败: {e}")

文件读取成功
列名: ['research_areas', 'journals', 'career_goal', 'skills']
前5行数据:
                       research_areas                        journals  \
0                           Marketing  Journal of Consumer Psychology   
1                                 NaN    Journal of Consumer Research   
2                                 NaN            Journal of Marketing   
3  Work and Organizational Psychology   Journal of Applied Psychology   
4                                 NaN            Personnel Psychology   

                        career_goal  \
0  marketing communications manager   
1                               NaN   
2                               NaN   
3        user experience researcher   
4                               NaN   

                                              skills  
0                                  CRM,SAP, Power BI  
1                                                NaN  
2                                                NaN  
3  usability,Qualtrics, qualitat

In [159]:
def load_research_data(file_path='D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/research_areas.xlsx',verbose=False):
    """
    加载研究领域数据
    """
    try:
        df = pd.read_excel(file_path)
        if verbose:
            print(f"读取成功，共{len(df)}行数据")
        research_data = {}

        current_research_area = None
        
        for index, row in df.iterrows():
            if verbose:
                print(f"正在处理第{index}行...")
            
            # 获取研究领域名称
            research_area = row['research_areas']
            if verbose:
                print(f"研究领域: {research_area}")

            # 如果当前行有研究领域名称，更新当前领域
            if pd.notna(research_area):
                current_research_area = research_area
                if verbose:
                    print(f"更新当前研究领域为：{current_research_area}")
                
            # 如果当前行没有研究领域名称，使用上一个非空领域
            elif pd.isna(research_area) and current_research_area is not None:
                research_area = current_research_area
                if verbose:
                    print(f"使用上一个研究领域：{research_area}")
            else:
                if verbose:
                    print("跳过空行（没有可用的研究领域）")
                continue

            # 如果是新研究领域，初始化数据结构
            if research_area not in research_data:
                # 新研究领域，初始化所有数据
                if verbose:
                    print(f"初始化新领域: {research_area}")
                research_data[research_area] = {
                    'journals': [],
                    'career_goal':'',
                    'skills': []
                }
                # 处理技能（只在第一行）
                if pd.notna(row['skills']):
                    skills_str = str(row['skills'])
                    skills_list = [skill.strip() for skill in skills_str.split(',') if skill.strip()]
                    research_data[research_area]['skills'] = skills_list
                    if verbose:
                        print(f"设置技能: {skills_list}")
                
                # 设置职业目标（只在第一行且为空时）
                if not research_data[research_area]['career_goal'] and pd.notna(row['career_goal']):
                    research_data[research_area]['career_goal'] = row['career_goal']
                    if verbose:
                        print(f"设置职业目标: {row['career_goal']}")
                    
            # 添加期刊（所有行）
            if pd.notna(row['journals']):
                research_data[research_area]['journals'].append(row['journals'])
                if verbose:
                    print(f"添加期刊: {row['journals']}")
                
        # 调试信息：打印每个领域的期刊数量
        if verbose:
            print("最终研究领域数量：", len(research_data))
        for area, data in research_data.items():
            if verbose:
                print("=" * 30)  # 分隔线，明确区分不同领域
                print(f"领域 {area} 的期刊数量：{len(data['journals'])}，期刊列表：{data['journals']}")
            
        return research_data
    except Exception as e:
        if verbose:
            print(f"加载失败：{e}")
        return {}

In [161]:
print("开始调用函数...")
result = load_research_data()  # 调用函数
print("函数执行完成，返回结果:", result)

开始调用函数...
函数执行完成，返回结果: {'Marketing': {'journals': ['Journal of Consumer Psychology', 'Journal of Consumer Research', 'Journal of Marketing'], 'career_goal': 'marketing communications manager', 'skills': ['CRM', 'SAP', 'Power BI']}, 'Work and Organizational Psychology': {'journals': ['Journal of Applied Psychology', 'Personnel Psychology', 'Journal of Occupational and Organizational Psychology'], 'career_goal': 'user experience researcher', 'skills': ['usability', 'Qualtrics', 'qualitative and quantitative UX research  methodologies']}, 'Finance': {'journals': ['Journal of Finance', 'Journal of Financial Economics', 'Review of Financial Studies'], 'career_goal': 'quantitative researcher', 'skills': ['python', 'SQL', 'C++', 'machine learning', 'back-testing']}}


In [163]:
def prepare_template_context(research_area, university, research_data):
    """
    为指定研究领域和大学准备模板上下文数据
    """
    # 检查研究领域是否存在
    if research_area not in research_data:
        print(f"未找到研究领域: {research_area}")
        return None
        
    # 获取该研究领域的数据
    area_data = research_data[research_area]
    
    # 准备期刊列表（取前3个）
    journals = area_data['journals'][:3]
    
    # 准备技能列表（取前3个）
    skills = area_data['skills']
    
    # 构建上下文字典
    context = {
        # 研究领域信息
        'research_area': research_area,
        'career_goal': area_data['career_goal'],
        'university': university,
        
        # 期刊信息
        'journal1': journals[0] if len(journals) > 0 else '',
        'journal2': journals[1] if len(journals) > 1 else '',
        'journal3': journals[2] if len(journals) > 2 else '',
        
        # 技能信息
        'skills': ', '.join(skills) if skills else ''
    }
    return context

In [165]:
# 测试数据
research_data = load_research_data()  # 先加载数据
context = prepare_template_context("Marketing", "哈佛大学", research_data)
print("生成的上下文数据：")
print(context)

生成的上下文数据：
{'research_area': 'Marketing', 'career_goal': 'marketing communications manager', 'university': '哈佛大学', 'journal1': 'Journal of Consumer Psychology', 'journal2': 'Journal of Consumer Research', 'journal3': 'Journal of Marketing', 'skills': 'CRM, SAP, Power BI'}


In [193]:
def safe_filename(text):
    """
    生成安全的文件名（移除特殊字符）
    """
    # 只保留字母、数字、空格、连字符和下划线
    safe_text = ''.join(c for c in text if c.isalnum() or c in (' ', '-', '_'))
    # 移除首尾空格
    return safe_text.strip()

# 测试上下文准备
print("\n测试上下文准备功能...")
research_data = load_research_data()
test_context = prepare_template_context('Marketing', 'Harvard University',research_data)

if test_context:
    print("上下文准备成功！")
    print("生成的上下文数据:")
    for key, value in test_context.items():
        print(f"  {key}: {value}")
else:
    print("上下文准备失败")


测试上下文准备功能...
上下文准备成功！
生成的上下文数据:
  research_area: Marketing
  career_goal: marketing communications manager
  university: Harvard University
  journal1: Journal of Consumer Psychology
  journal2: Journal of Consumer Research
  journal3: Journal of Marketing
  skills: CRM, SAP, Power BI


In [197]:
# 单个申请信生成函数
def generate_single_application(template_path, output_dir, context, convert_pdf=True):
    """
    生成单个申请信文档
    """
    try:
        # 1. 检查模板文件是否存在
        if not os.path.exists(template_path):
            print(f"模板文件不存在: {template_path}")
            return None
        
        # 2. 加载Word模板
        doc = DocxTemplate(template_path)
        
        # 3. 使用上下文数据渲染模板
        doc.render(context)
        
        # 4. 生成安全的文件名
        safe_uni = safe_filename(context['university'])
        safe_area = safe_filename(context['research_area'])
        filename = f"SOP_{safe_uni}_{safe_area}.docx"
        output_path = os.path.join(output_dir, filename)
        
        # 5. 确保输出目录存在
        os.makedirs(output_dir, exist_ok=True)
        
        # 6. 保存Word文档
        doc.save(output_path)
        print(f"✅ 生成成功: {filename}")
        
        # 7. 转换为PDF（如果支持）
        if convert_pdf and PDF_SUPPORT:
            try:
                pdf_filename = filename.replace('.docx', '.pdf')
                pdf_path = os.path.join(output_dir, pdf_filename)
                convert(output_path, pdf_path)
                print(f"PDF转换成功: {pdf_filename}")
            except Exception as e:
                print(f"PDF转换失败: {e}")
        
        return output_path
        
    except Exception as e:
        print(f"生成申请信失败: {e}")
        return None

In [207]:
# 测试单个申请信生成
print("\n测试单个申请信生成...")
try:
    from docx2pdf import convert
    PDF_SUPPORT = True
    print("PDF转换支持已启用")
except ImportError:
    PDF_SUPPORT = False
    print("PDF转换支持不可用，将只生成Word文档")
if test_context:
    # 创建输出目录
    os.makedirs('D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/单份申请信输出', exist_ok=True)
    
    # 测试生成（假设模板文件在templates目录）
    test_result = generate_single_application(
        template_path='D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/word_template.docx', 
        output_dir='D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/单份申请信输出', 
        context=test_context,
        convert_pdf=True  
    )
    
    if test_result:
        print(f"测试文件已生成: {test_result}")
    else:
        print("请先创建Word模板文件: templates/template.docx")


测试单个申请信生成...
PDF转换支持已启用
✅ 生成成功: SOP_Harvard University_Marketing.docx


  0%|          | 0/1 [00:00<?, ?it/s]

PDF转换成功: SOP_Harvard University_Marketing.pdf
测试文件已生成: D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/单份申请信输出\SOP_Harvard University_Marketing.docx


In [209]:
# 批量生成所有申请信
def generate_all_applications(template_path, output_dir, research_data, universities):
    """
    批量生成所有大学和研究领域的申请信
    """
    # 检查必要数据是否已加载
    if not universities or not research_data:
        print("数据未正确加载，请先运行数据加载单元格")
        return []
    try:
        from docx2pdf import convert
        PDF_SUPPORT = True
    except ImportError:
        PDF_SUPPORT = False
    
    print("开始批量生成申请信...")
    print("=" * 60)
    
    # 计算总数量
    total_count = len(universities) * len(research_data)
    print(f"📊 生成计划:")
    print(f"  大学数量: {len(universities)}")
    print(f"  研究领域: {len(research_data)}")
    print(f"  总申请信: {total_count} 份")
    
    successful_generations = []
    current_count = 0
    
    # 主循环：遍历所有大学和研究领域的组合
    for university in universities:
        print(f"\n处理大学: {university}")
        
        for research_area in research_data.keys():
            current_count += 1
            print(f"生成 ({current_count}/{total_count}): {research_area}")
            
            # 准备模板上下文
            context = prepare_template_context(research_area, university, research_data)
            
            if context:
                # 生成单个申请信
                result_path = generate_single_application(
                    template_path, 
                    output_dir, 
                    context, 
                    convert_pdf=False
                )
                
                if result_path:
                    successful_generations.append(result_path)
    
    # 生成报告
    print("\n" + "=" * 60)
    print("批量生成完成！")
    print(f"成功生成: {len(successful_generations)}/{total_count} 份申请信")
    print(f"输出目录: {os.path.abspath(output_dir)}")
    
    # 显示生成的文件列表
    if successful_generations:
        print("\n📋 生成的文件列表（前5个）:")
        for i, filepath in enumerate(successful_generations[:5], 1):
            filename = os.path.basename(filepath)
            print(f"  {i}. {filename}")
        
        if len(successful_generations) > 5:
            print(f"  ... 和 {len(successful_generations) - 5} 个更多文件")
    
    return successful_generations

In [187]:
# 1. 加载研究数据
research_data = load_research_data()
universities = load_universities_data()
# 2. 批量生成申请信
result_files = generate_all_applications(
    template_path='D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/word_template.docx',      # 指定Word模板路径
    output_dir='D:/大学学习/大三上/人工智能与python （量化）/作业代码/homework_2/申请信输出',                 
    research_data=research_data,
    universities=universities
)

print(f"批量生成完成，共生成{len(result_files)}份申请信")

✅ 成功加载 30 所大学
📋 大学列表预览:
  1. Harvard University
  2. Massachusetts Institute of Technology (MIT)
  3. University of Chicago
  4. Princeton University
  5. Stanford University
  ... 和 25 所更多大学
开始批量生成申请信...
📊 生成计划:
  大学数量: 30
  研究领域: 3
  总申请信: 90 份

处理大学: Harvard University
生成 (1/90): Marketing
✅ 生成成功: SOP_Harvard University_Marketing.docx
生成 (2/90): Work and Organizational Psychology
✅ 生成成功: SOP_Harvard University_Work and Organizational Psychology.docx
生成 (3/90): Finance
✅ 生成成功: SOP_Harvard University_Finance.docx

处理大学: Massachusetts Institute of Technology (MIT)
生成 (4/90): Marketing
✅ 生成成功: SOP_Massachusetts Institute of Technology MIT_Marketing.docx
生成 (5/90): Work and Organizational Psychology
✅ 生成成功: SOP_Massachusetts Institute of Technology MIT_Work and Organizational Psychology.docx
生成 (6/90): Finance
✅ 生成成功: SOP_Massachusetts Institute of Technology MIT_Finance.docx

处理大学: University of Chicago
生成 (7/90): Marketing
✅ 生成成功: SOP_University of Chicago_Marketing.docx
生成 (8/90): Wor